<a href="https://colab.research.google.com/github/nickolasnikolic/jupyter/blob/main/music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mido

     |████████████████████████████████| 51 kB 4.2 MB/s 


In [12]:
from mido import Message, MidiFile, MidiTrack
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
import random

#read signals
source = [
    MidiFile('./data/symphony_9_1.mid'), 
    #MidiFile('./data/symphony_9_2.mid'),
    #MidiFile('./data/symphony_9_3.mid'),
    #MidiFile('./data/symphony_9_4.mid')
]
fileCount = 0
for file in source:
    metadata = []
    originalnotelist = []
    randomnotespool = []
    trackCount = 0
    tracknames = []
    
    for i, track in enumerate(file.tracks):
        if trackCount % 25 == 0:
            print("track " + str(trackCount) + ' through ' + str(trackCount + 25) + ' being read')
        tracknames.append(track)
        trackCount += 1
        for m in track:
            if (m.type == 'note_on' or m.type == 'note_off'):
                originalnotelist.append({ 'track': trackCount, 'type': m.type, 'channel': m.channel, 'note': m.note, 'time': m.time, 'velocity': m.velocity })
    
    #learn signals
    print('prepping data')
    notes = pd.DataFrame.from_records(originalnotelist)
    notesfortraining = notes.filter(['track_name', 'note', 'channel', 'time', 'velocity'])
    big_X, little_y = train_test_split(notesfortraining, test_size=0.5)

    # Create and fit a nearest-neighbor classifier
    regression = KNeighborsRegressor(n_neighbors=4)
    print('regression fitting')
    regression.fit(big_X, little_y)

    #predict
    predictednotes = []
    candidates = regression.predict(big_X)
    predictednotes.append(candidates)
    print(str(len(predictednotes[0])) + ' notes predicted...' )

    midifile = MidiFile()
    randomMIDIfile = MidiFile()
    notenumber = 0
    #write predictions to file        
    for name in tracknames:
      trackTemp = []
      for t in range(0, trackCount):
        if t % 5 == 0:
          print("track " + str(t) + ' being written')
        
        print('placing metamessage')
        for metamessage in name:
          trackTemp.append(metamessage)

        print('enterting predicted notes')
        for note in predictednotes[0]:
            print('track', t, 'Note number', notenumber, note)
            notenumber += 1
            
            note = note.round(0)
            note = note.astype(int)

            trackTemp.append( Message( 'note_on', note=note[0],  channel=min(note[1], 16),  time=max(note[2], 0), velocity=max(note[3], 127) ))

    print('writing to file ' + str(fileCount))
    fileCount += 1 
    midifile.save('./data/Beethoven Approximation - Movement ' + str(fileCount) + '.midi')

print('\nyour music is ready!')

Streaming output truncated to the last 5000 lines.
track 14 Note number 6955378 [67.75  6.   21.25 44.25]
track 14 Note number 6955379 [70.75  9.5  89.25 34.  ]
track 14 Note number 6955380 [66.5   6.5  34.25 56.75]
track 14 Note number 6955381 [67.  11.5 20.  37.5]
track 14 Note number 6955382 [61.25  4.   30.5  91.  ]
track 14 Note number 6955383 [52.25 10.25 94.75 56.75]
track 14 Note number 6955384 [54.75 11.5  44.75 29.75]
track 14 Note number 6955385 [ 52.75  11.25 147.75  31.75]
track 14 Note number 6955386 [68.    7.5  94.75 44.25]
track 14 Note number 6955387 [ 70.5    5.25 264.75  56.75]
track 14 Note number 6955388 [64.25 12.5   3.25 35.75]
track 14 Note number 6955389 [52.  13.   6.  99.5]
track 14 Note number 6955390 [63.25 10.25 46.75 12.5 ]
track 14 Note number 6955391 [ 71.25   7.25 106.75  89.  ]
track 14 Note number 6955392 [ 58.25   7.75 115.25  25.  ]
track 14 Note number 6955393 [61.   10.   14.5  31.75]
track 14 Note number 6955394 [52.25 10.25 94.75 56.75]
track 